# Multi-Branch Interactions & Patterns

This notebook demonstrates how to orchestrate multiple AI agents working together, showcasing the power of lionagi's branch system for complex AI workflows.

**What you'll learn:**
- Creating separate AI agents with distinct personalities and roles
- Passing context between different branches for collaboration
- Implementing feedback loops for iterative improvement
- Persisting and reloading conversation state

In [1]:
from lionagi import Branch
from IPython.display import Markdown

In [2]:
sys_comedian = "As a comedian, you are sarcastically funny"

prompt1 = "short joke: a blue whale and a big shark meet at the bar and start dancing"
prompt2 = "continue the joke: and then they stopped"

In [3]:
# start a LLM session and have consecutive conversation with it
comedian = Branch(system=sys_comedian)

joke1 = await comedian.communicate(prompt1)
joke2 = await comedian.communicate(prompt2)

In [4]:
Markdown(joke1)

So this blue whale and a big shark walk into a bar, see each other, and just start dancing. I guess even sea monsters know it’s time to shake a fin—because nothing says “genuine connection” like synchronized swimming in a speakeasy!

In [5]:
Markdown(joke2)

And then they stopped—turns out even giants and predators have their limits. Apparently, the dance floor wasn’t big enough for a whale-sized twirl and a shark-sized splash!

**Introducing Multi-Agent Collaboration:**

Now we'll create a second AI agent with a different role and personality. This demonstrates a common pattern in AI applications: having specialized agents that excel at different tasks.

**The Multi-Agent Pattern:**
- **Comedian**: Creative, humorous, focused on entertainment
- **Critic**: Analytical, logical, focused on evaluation and improvement

This separation of concerns allows each agent to specialize in what they do best, leading to higher quality results through collaboration.

In [6]:
# some messages for the critic
sys_critic = "you are a respected commentator, you are artistically logical"

prompt3 = "short comments, what do you think about the first joke?"
prompt4 = (
    "provide a concise artistic critique on both jokes, and rate from 1-10"
)

In [7]:
# create a session with the critic and generate some comments in a multi-turn conversation
critic = Branch(system=sys_critic)

# provide context for the critic, context needs to be string or string mappings/iterables
context = {
    "joke1": {"prompt": prompt1, "response": joke1},
    "joke2": {"prompt": prompt2, "response": joke2},
}

# you only need to provide same context once in a conversation
comment1 = await critic.communicate(prompt3, context=context)
comment2 = await critic.communicate(prompt4)

In [8]:
Markdown(comment1)

The first joke cleverly paints a whimsical underwater scene, anthropomorphizing sea creatures in a human setting. It delights with its playful imagery and sets a light, humorous tone—showing that even the ocean’s giants can join in the rhythm of fun.

In [9]:
Markdown(comment2)

**First joke critique and rating:**  
The joke's charm lies in its imaginative scenario, blending marine life with human social habits. Its playful tone and vivid imagery create a delightful, visual humor that engages the audience. I’d rate it an 8 for creativity and lightheartedness.

**Second joke critique and rating:**  
This continuation offers a clever, humorous twist that adds depth to the initial scene. The punchline’s surprise highlights the limitations of even fantastical characters, grounding the whimsy with a touch of realism. I’d rate it a 7 for wit and effective timing.

**Implementing Feedback Loops:**

One of the most powerful patterns in AI workflows is the feedback loop, where one agent's output becomes another agent's input. This creates an iterative improvement cycle.

**The Feedback Process:**
1. **Initial Creation**: Comedian generates content
2. **Critical Analysis**: Critic evaluates the content  
3. **Reflection & Improvement**: Comedian receives feedback and refines their work

This pattern mimics real-world creative processes and often leads to significantly improved results compared to single-shot generation.

In [10]:
# messages for comedian again
prompt5 = (
    "your jokes were evaluated by a critic, does it make sense to you? why?"
)
prompt6 = "basing on your reflection, write joke1 again"
prompt7 = "write joke2 again"

# let us provide critic's feedback to comedian
context2 = [i.response for i in critic.msgs.assistant_responses]

In [11]:
# run the instructions
reflect = await comedian.communicate(prompt5, context=context2)
joke11 = await comedian.communicate(prompt6)
joke22 = await comedian.communicate(prompt7)

In [12]:
Markdown(reflect)

Yes, it makes sense to me because both jokes play on the contrast between the fantastical and the mundane—underwater creatures acting like humans in a bar, then suddenly revealing their limitations. It’s a classic setup and punchline structure with a fun twist: even monsters have boundaries. The humor's effectiveness comes from the vivid imagery and the unexpected ending, which feels both whimsical and relatable. So, in short, if a critic appreciates playful imagination with a dash of irony, then these jokes totally deliver.

In [13]:
Markdown(joke11)

# Assistant Response

**Joke 1 (Rewritten)**:
So a blue whale and a big shark walk into a bar, start dancing like they’re the life of the sea. Turns out even ocean giants love to get their groove on — because nothing beats a little aquatic rhythm, right?

In [14]:
Markdown(joke22)

# Assistant Response

**Joke 2 (Rewritten)**:
And then they stopped—because apparently, even the biggest sea creatures hit their limits. Turns out, there’s only so much room for a whale to twirl and a shark to splash before they realize the dance floor wasn’t quite big enough for their aquatic talents.

In [15]:
comedian.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,1.753071e+09,system,"{'system_message': 'As a comedian, you are sar...",5e2b56cc-670e-41cb-a2ae-57f1232d3265,system,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,{'lion_class': 'lionagi.protocols.messages.sys...
1,1.753071e+09,user,"{'context': [], 'instruction': 'short joke: a ...",07b15902-36bc-4bda-90be-ac9fb1f1d0ad,user,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,{'lion_class': 'lionagi.protocols.messages.ins...
2,1.753071e+09,assistant,{'assistant_response': 'So this blue whale and...,a1b400d1-5b60-4642-9e38-1b9465acbbb0,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,user,{'model_response': {'id': 'chatcmpl-BvcCdiScKg...
3,1.753071e+09,user,"{'context': [], 'instruction': 'continue the j...",797b5616-28b2-450b-976d-37b5e1ece9ab,user,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,{'lion_class': 'lionagi.protocols.messages.ins...
4,1.753071e+09,assistant,{'assistant_response': 'And then they stopped—...,7ba13ce4-601f-4847-972f-82e7c9690d5d,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,user,{'model_response': {'id': 'chatcmpl-BvcCei6YgY...
5,1.753071e+09,user,{'context': ['The first joke cleverly paints a...,e89f5d93-1501-45e3-a4c1-36a89c8d7f57,user,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,{'lion_class': 'lionagi.protocols.messages.ins...
6,1.753071e+09,assistant,"{'assistant_response': 'Yes, it makes sense to...",f764b34a-a357-433d-ab29-4fb5261716e0,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,user,{'model_response': {'id': 'chatcmpl-BvcCgFlbuG...
7,1.753071e+09,user,"{'context': [], 'instruction': 'basing on your...",0d4b0561-611e-4130-8490-69fd531cb4e3,user,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,{'lion_class': 'lionagi.protocols.messages.ins...
8,1.753071e+09,assistant,{'assistant_response': '# Assistant Response ...,5436d17c-e9d3-4754-8eaf-6a25ef22cd50,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,user,{'model_response': {'id': 'chatcmpl-BvcChNT900...
9,1.753071e+09,user,"{'context': [], 'instruction': 'write joke2 ag...",4b1bafc7-3904-4716-8471-390cae403072,user,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,{'lion_class': 'lionagi.protocols.messages.ins...


In [16]:
critic.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,1.753071e+09,system,{'system_message': 'you are a respected commen...,ba667b0f-203d-4ede-af0d-139746f91318,system,5f8e67e3-2242-43c2-a106-eb86819d3012,{'lion_class': 'lionagi.protocols.messages.sys...
1,1.753071e+09,user,{'context': [{'joke1': {'prompt': 'short joke:...,9a5e6cca-9d54-414d-ab4d-04d43063581d,user,5f8e67e3-2242-43c2-a106-eb86819d3012,{'lion_class': 'lionagi.protocols.messages.ins...
2,1.753071e+09,assistant,{'assistant_response': 'The first joke cleverl...,3bc27fc1-5a90-4fc0-aa9e-28cda5892ca1,5f8e67e3-2242-43c2-a106-eb86819d3012,user,{'model_response': {'id': 'chatcmpl-BvcCelVG2J...
3,1.753071e+09,user,"{'context': [], 'instruction': 'provide a conc...",25413925-eda6-4938-bb23-716dfac047e3,user,5f8e67e3-2242-43c2-a106-eb86819d3012,{'lion_class': 'lionagi.protocols.messages.ins...
4,1.753071e+09,assistant,{'assistant_response': '**First joke critique ...,340ef3d9-195d-48d2-aecb-b326d279dd20,5f8e67e3-2242-43c2-a106-eb86819d3012,user,{'model_response': {'id': 'chatcmpl-BvcCfKvuj6...


**Branch Persistence: Saving & Loading State**

One of lionagi's powerful features is the ability to serialize and persist entire conversation branches. This enables:

**Production Use Cases:**
- **Session Recovery**: Resume conversations after system restarts
- **Conversation Archiving**: Store important dialogues for future reference  
- **A/B Testing**: Save different conversation branches for comparison
- **Audit Trails**: Maintain complete records of AI interactions

The serialization captures not just the messages, but the entire conversation state including metadata, model responses, and conversation flow.

In [ ]:
import json

with open("data/002_comedian.json", "w") as f:
    json.dump(comedian.to_dict(), f, indent=4)

with open("data/002_critic.json", "w") as f:
    json.dump(critic.to_dict(), f, indent=4)

In [ ]:
# To use them again

with open("data/002_comedian.json") as f:
    dict_ = json.load(f)
    comedian = Branch.from_dict(dict_)

comedian.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,1.753071e+09,system,{'system_message': {'id': '5e2b56cc-670e-41cb-...,bb7f8bf8-0485-4411-a76d-5db2e21d4d5c,system,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,{'lion_class': 'lionagi.protocols.messages.sys...
1,1.753071e+09,system,"{'system_message': 'As a comedian, you are sar...",5e2b56cc-670e-41cb-a2ae-57f1232d3265,system,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,{'lion_class': 'lionagi.protocols.messages.sys...
2,1.753071e+09,user,"{'context': [], 'instruction': 'short joke: a ...",07b15902-36bc-4bda-90be-ac9fb1f1d0ad,user,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,{'lion_class': 'lionagi.protocols.messages.ins...
3,1.753071e+09,assistant,{'assistant_response': 'So this blue whale and...,a1b400d1-5b60-4642-9e38-1b9465acbbb0,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,user,{'model_response': {'id': 'chatcmpl-BvcCdiScKg...
4,1.753071e+09,user,"{'context': [], 'instruction': 'continue the j...",797b5616-28b2-450b-976d-37b5e1ece9ab,user,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,{'lion_class': 'lionagi.protocols.messages.ins...
5,1.753071e+09,assistant,{'assistant_response': 'And then they stopped—...,7ba13ce4-601f-4847-972f-82e7c9690d5d,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,user,{'model_response': {'id': 'chatcmpl-BvcCei6YgY...
6,1.753071e+09,user,{'context': ['The first joke cleverly paints a...,e89f5d93-1501-45e3-a4c1-36a89c8d7f57,user,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,{'lion_class': 'lionagi.protocols.messages.ins...
7,1.753071e+09,assistant,"{'assistant_response': 'Yes, it makes sense to...",f764b34a-a357-433d-ab29-4fb5261716e0,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,user,{'model_response': {'id': 'chatcmpl-BvcCgFlbuG...
8,1.753071e+09,user,"{'context': [], 'instruction': 'basing on your...",0d4b0561-611e-4130-8490-69fd531cb4e3,user,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,{'lion_class': 'lionagi.protocols.messages.ins...
9,1.753071e+09,assistant,{'assistant_response': '# Assistant Response ...,5436d17c-e9d3-4754-8eaf-6a25ef22cd50,a8fd8a27-4c7e-4a7a-ae4e-9f8b23d24192,user,{'model_response': {'id': 'chatcmpl-BvcChNT900...
